In [1]:
# Libraries
import sqlite3
import json
import os
import pandas as pd
from datetime import datetime as dt
import numpy as np
from datetime import time as t

In [2]:
# Sources
user_credential = r'\Users\dinhhoang.nguyen.CONCENTRIX\OneDrive - Concentrix Corporation\WFM-internal'

eps_path = os.path.join("C:", user_credential, r"Data\EPS Tableau\EPS Query")
eps_full_path = os.path.join("C:", user_credential, r"Data\EPS Tableau")

cpi_path = os.path.join("C:", user_credential, r"Data\Ticket\CPI Query")
cpi_full_path = os.path.join("C:", user_credential, r"Data\Ticket")

aht_path = os.path.join("C:", user_credential, r"Data\AHT\AHT query")
aht_full_path = os.path.join("C:", user_credential, r"Data\AHT")

csat_path = os.path.join("C:", user_credential, r"Data\CSAT RAW\CSAT Query")
csat_full_path = os.path.join("C:", user_credential, r"Data\CSAT RAW")

csatreso_path = os.path.join("C:", user_credential, r"Data\CSAT Reso\CSAT Reso Query")
csatreso_full_path = os.path.join("C:", user_credential, r"Data\CSAT Reso")

cuic_path = os.path.join("C:", user_credential, r"Data\CUIC\CUIC Query")
cuic_full_path = os.path.join("C:", user_credential, r"Data\CUIC")

exception_path = os.path.join("C:", user_credential, r"Data\Exception")
extension_path = os.path.join("C:", user_credential, r"Data\Extension")
iex_path = os.path.join("C:", user_credential, r"Data\IEX")
label_path = os.path.join("C:", user_credential, r"Data\Labels")
ioshrinkage_path = os.path.join("C:", user_credential, r"Data\IO Shrinkage")

ot_path = os.path.join("C:", user_credential, r"Data\OT\OT Query")
ot_full_path = os.path.join("C:", user_credential, r"Data\OT")

psat_path = os.path.join("C:", user_credential, r"Data\PSAT RAW\PSAT Query")
psat_full_path = os.path.join("C:", user_credential, r"Data\PSAT RAW\PSAT Query")

quality_path = os.path.join("C:", user_credential, r"Data\Quality.v2\Quality query")
quality_full_path = os.path.join("C:", user_credential, r"Data\Quality.v2")

ramco_path = os.path.join("C:", user_credential, r"Data\Ramco\Ramco Query")
ramco_full_path = os.path.join("C:", user_credential, r"Data\Ramco")

ramcoot_path = os.path.join("C:", user_credential, r"Data\Ramco OT\Ramco OT Query")
ramcoot_full_path = os.path.join("C:", user_credential, r"Data\Ramco OT")

requirement_path = os.path.join("C:", user_credential, r"Data\Requirement")

schedule_path = os.path.join("C:", user_credential, r"Data\Schedule 2\Schedule Query")
schedule_full_path = os.path.join("C:", user_credential, r"Data\Schedule 2")

tl_path = os.path.join("C:", user_credential, r"Data\TL")
training_path = os.path.join("C:", user_credential, r"Data\Training")
masterroster_path = os.path.join("C:", user_credential, r"Data\User")
workplan_path = os.path.join("C:", user_credential, r"Data\Workplan\Workplan Query")

rona_path = os.path.join("C:", user_credential, r"Data\RONA daily tracking\RONA query")
rona_full_path = os.path.join("C:", user_credential, r"Data\RONA daily tracking")

workplan_wst_path = os.path.join("C:", user_credential, r"Data\Workplan wst\Workplan wst Query")

In [3]:
time_reset = dt(year=2023, month=8, day=1)
update_less = False

In [4]:
#FUNCTION TO IMPORT FOLDERS
def import_csv(path):
    raw = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            file_path = os.path.join(path, file)
            try:
                file_data = pd.read_csv(file_path)
            except:
                file_data = pd.read_csv(file_path, encoding='latin-1')
            file_data["filename"] = file
            raw.append(file_data)
            final_raw = pd.concat(raw, ignore_index=True)
    return final_raw
    
def import_xlsx(path, sheet_name):
    raw = []
    for file in os.listdir(path):
        if file.endswith(".xlsx"):
            file_path = os.path.join(path, file)
            try:
                file_data = pd.read_excel(file_path, engine="openpyxl", sheet_name = sheet_name)
            except:
                print(f"error with {file_path}")
            file_data["filename"] = file
            raw.append(file_data)
            final_raw = pd.concat(raw, ignore_index=True)
    return final_raw

In [5]:
# Connect to Database
conn = sqlite3.connect(os.path.join(r'C:', user_credential, r'DB\bkn.db'))
cursor = conn.cursor()

In [6]:
# IMPORT MASTER ROSTER
masterroster = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\User\CNX Global Master Roster.xlsx'), sheet_name="Sheet1")
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("Int64")
# export master roster to json
masterroster['PST_Start_Date'] = pd.to_datetime(masterroster['PST_Start_Date'], format='mixed').dt.strftime('%Y-%m-%d')
masterroster['LWD'] = pd.to_datetime(masterroster['LWD'], format='mixed').dt.strftime('%Y-%m-%d')
masterroster['Termination_Date'] = pd.to_datetime(masterroster['Termination_Date'], format='mixed').dt.strftime('%Y-%m-%d')
masterroster = masterroster.loc[masterroster['Role']=='Agent'] #since Ted  column contains duplicated values (agents and hierarchy have the same names), we have to filter Agent role to have it removed duplicate
masterroster.to_sql('masterroster', conn, if_exists='replace', index=False)



688

In [7]:
# IMPORT EPS_full
if update_less is False:
    eps_full = import_csv(path = eps_full_path)
else:
    eps_full = import_csv(path = eps_path)

eps_full = eps_full.drop(columns=['Index', 'filename', 'Date'])
# eps_full['Session Login'] = pd.to_datetime(eps_full['Session Login'], errors='coerce')
# eps_full['Session Login'] = eps_full['Session Login'].dt.tz_localize('Europe/Berlin', ambiguous=True).dt.tz_convert('UTC')
# eps_full['Session Logout'] = pd.to_datetime(eps_full['Session Logout'], errors='coerce')
# eps_full['Session Logout'] = eps_full['Session Logout'].dt.tz_localize('Europe/Berlin', ambiguous=True).dt.tz_convert('UTC')
 
eps_full['Session login VN'] = pd.to_datetime(eps_full['Session Login'], errors='coerce').dt.tz_localize('Europe/Berlin', ambiguous=True).dt.tz_convert('UTC') + pd.Timedelta(hours=7)
eps_full['Session logout VN'] = pd.to_datetime(eps_full['Session Logout'], errors='coerce').dt.tz_localize('Europe/Berlin', ambiguous=True).dt.tz_convert('UTC') + pd.Timedelta(hours=7)
eps_full['Session login date'] = pd.to_datetime(eps_full['Session login VN']).dt.strftime('%Y-%m-%d')
eps_full['Session login time'] = pd.to_datetime(eps_full['Session login VN']).dt.time
eps_full['Session logout date'] = pd.to_datetime(eps_full['Session logout VN']).dt.strftime('%Y-%m-%d')
eps_full['Session logout time'] = pd.to_datetime(eps_full['Session logout VN']).dt.time

eps_full['Session Login'] = pd.to_datetime(eps_full['Session Login']).dt.strftime('%Y-%m-%d %I:%M:%S %p')
eps_full['Session Logout'] = pd.to_datetime(eps_full['Session Logout']).dt.strftime('%Y-%m-%d %I:%M:%S %p')
eps_full = eps_full.drop(columns=['Session login VN', 'Session logout VN', 'Username', 'manager_username', 'sitecode', 'Session Time'])

# export eps_full_full to json
eps_full.to_sql('eps', conn, if_exists='replace', index=False)

875108

In [8]:
 #IMPORT AHT
if update_less is False:
    aht_full = import_csv(path = aht_full_path)
else:
    aht_full = import_csv(path = aht_path)

aht_full = aht_full.drop(columns=['filename'])
masterroster_aht_full = masterroster[['Employee_ID', 'TED Name']]
aht_full = pd.merge(aht_full, masterroster_aht_full, left_on='Staff', right_on ='TED Name', how='left')
aht_full = aht_full.drop(columns=['TED Name'])
aht_full['Date'] = pd.to_datetime(aht_full['Date'], errors='coerce')

aht_full.to_sql('aht', conn, if_exists='replace', index=False)

C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23636\4180394810.py:8: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  file_data = pd.read_csv(file_path)
C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23636\4180394810.py:8: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  file_data = pd.read_csv(file_path)
C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23636\4180394810.py:8: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  file_data = pd.read_csv(file_path)


1958862

In [9]:
#IMPORT CSAT
if update_less is False:
    csat = import_csv(path = csat_full_path)
else:
    csat = import_csv(path = csat_path)

masterroster_csat = masterroster[['Employee_ID', 'TED Name']]
csat['Date '] = pd.to_datetime(csat['Date '], format='%m/%d/%Y')
csat = csat.rename(columns={'Date ': 'Date'})
csat = pd.merge(csat, masterroster_csat, left_on='Staff', right_on ='TED Name', how='left')
csat = csat.drop(columns=['Sort by Dimension', 'Has Comment', '"Comment"', 'Reservation Link', 'View comment', 'Sort by Dimension (copy)', 'filename', 'TED Name', 'Max. Sort by Dimension'])
csat['Date'] = csat['Date'].dt.strftime('%Y-%m-%d')
# Export to SQL
csat.to_sql('csat', conn, if_exists='replace', index=False)

1958862

In [10]:
#IMPORT CSAT_RESO
if update_less is False:
    csat_reso = import_csv(path = csatreso_full_path)
else:
    csat_reso = import_csv(path = csatreso_path)
    
masterroster_csat_reso = masterroster[['Employee_ID', 'TED Name']]
csat_reso['Date '] = pd.to_datetime(csat_reso['Date '], errors='coerce')
csat_reso = csat_reso.rename(columns={'Date ': 'Date'})
csat_reso = pd.merge(csat_reso, masterroster_csat_reso, left_on='Staff', right_on ='TED Name', how='left')
csat_reso = csat_reso.drop(columns=['Sort by Dimension', 'Has Comment', '"Comment"', 'Reservation Link', 'View comment', 'Sort by Dimension (copy)', 'filename', 'TED Name', 'Max. Sort by Dimension'])
csat_reso['Date'] = csat_reso['Date'].dt.strftime('%Y-%m-%d')

#export csat_reso to sql
csat_reso.to_sql('csat_reso', conn, if_exists='replace', index=False)

21126

In [11]:
# IMPORT RAMCO
if update_less is False:
    ramco = import_csv(path = ramco_full_path)
else:
    ramco = import_csv(path = ramco_path)

ramco['Attribute'] = pd.to_datetime(ramco['Attribute'], errors='coerce').dt.date
ramco=ramco.rename(columns={'Attribute':'Date'})
ramco['EID']=ramco['EID'].astype('Int64')

# export ramco to json
ramco.to_sql('ramco', conn, if_exists='replace', index=False)

889180

In [12]:
# IMPORT SCHEDULE
if update_less is False:
    schedule = import_xlsx(path = schedule_full_path,sheet_name='Sheet1')
else:
    schedule = import_xlsx(path = schedule_path,sheet_name='Sheet1')

schedule['Attribute'] = pd.to_datetime(schedule['Attribute'], errors='coerce').dt.date
schedule=schedule.iloc[:,0:8]
condition=(schedule['Value']=='OFF')|(schedule['Value']=='AL')|(schedule['Value']=='CO')|(schedule['Value']=='UPL')|(schedule['Value']=='VGH')
condition_training=(schedule['Value']=='Training')|(schedule['Value']=='PEGA')
schedule['Shift'] = np.where(condition, 'OFF', schedule['NS Check'])
schedule['Shift'] = np.where(condition_training, 'Training', schedule['Shift'])
schedule=schedule[['Emp ID','Name','Attribute','Value','LOB','Shift']]
schedule=schedule.rename(columns={'Attribute':'Date','Value':'Shift','Shift':'Shift_type'})
schedule['Emp ID'] = schedule['Emp ID'].astype("Int64")
 
week_shift=schedule[['Emp ID','Date','Shift']]
week_shift['Date'] = pd.to_datetime(week_shift['Date'],errors='coerce')
week_shift['Week'] = week_shift['Date'].dt.strftime('%Y%W')
week_shift=week_shift[['Emp ID','Week','Shift']]
week_shift = week_shift[week_shift['Shift'].str.contains('00$')]
week_shift=week_shift.groupby(['Emp ID','Week','Shift'],as_index=False).count()
week_shift=week_shift.groupby(['Emp ID','Week'],as_index=False).agg(min=('Shift','min'))
week_shift=week_shift.rename(columns={'min':'Week_shift'})
 
schedule['Date'] = pd.to_datetime(schedule['Date'],errors='coerce')
schedule['Week'] = schedule['Date'].dt.strftime('%Y%W')
schedule=pd.merge(schedule,week_shift,left_on=['Emp ID','Week'],right_on=['Emp ID','Week'],how='left')

schedule['Date']=schedule['Date'].dt.date

schedule=pd.merge(schedule,ramco,left_on=['Emp ID','Date'],right_on=['EID','Date'],how='left')
def actualshift_func(x):
    if x['Value']=='PH' or x['Value']=='PO' :
        return x['Week_shift']
    else:         
        return x['Shift']
schedule['Actual_shift'] = schedule.apply(actualshift_func, axis=1)
def actualshift1_func(x):
    if x['Actual_shift'] in ['0400-1300','0500-1400','0600-1500','0700-1600','0800-1700','0900-1800','1000-1900','1100-2000',
                      '1200-2100','1300-2200','1400-2300','1500-2400','1600-0100','1700-0200', '1800-0300','1900-0400']:     
        return 'DS'
    elif x['Actual_shift'] in ['2000-0500','2100-0600','2200-0700']:
        return 'NS'
    else:      
        return x['Shift_type']
schedule['Actual_shift_type'] = schedule.apply(actualshift1_func, axis=1)
schedule=schedule[['Emp ID','Name','Date','Actual_shift','LOB','Actual_shift_type']]
schedule=schedule.rename(columns={'Actual_shift':'Shift','Actual_shift_type':'Shift_type'}) # type: ignore

# export schedule to json
schedule['Date'] = pd.to_datetime(schedule['Date'], errors='coerce').dt.strftime('%Y-%m-%d')
schedule.to_sql('schedule', conn, if_exists='replace', index=False)

C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23636\2835404172.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_shift['Date'] = pd.to_datetime(week_shift['Date'],errors='coerce')
C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23636\2835404172.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_shift['Week'] = week_shift['Date'].dt.strftime('%Y%W')


152921

In [13]:
#IMPORT CUIC
if update_less is False:
    cuic = import_xlsx(path = cuic_full_path,sheet_name="Khoa Interval Attendance Check-")
else:
    cuic = import_xlsx(path = cuic_path,sheet_name="Khoa Interval Attendance Check-")

cuic['Date'] = pd.to_datetime(cuic['Interval'], errors='coerce').dt.date
cuic['Time'] = pd.to_datetime(cuic['Interval'], errors='coerce').dt.time
cuic = pd.merge(cuic, masterroster, left_on='LoginName', right_on ='Booking Login ID', how='left')
cuic=cuic.iloc[:,0:9]

schedule['Date'] = pd.to_datetime(schedule['Date'], errors='coerce').dt.date

cuic = pd.merge(cuic, schedule, left_on=['Employee_ID','Date'], right_on =['Emp ID','Date'], how='left')
cuic=cuic[['FullName','LoginName','Interval','AgentAvailTime','AgentLoggedOnTime','Date','Time','Employee_ID','Shift_type','Shift']]
cuic['Date-1']=pd.to_datetime(cuic['Date'], errors='coerce') - pd.Timedelta(1,"d")
cuic['Date-1']=pd.to_datetime(cuic['Date-1'], errors='coerce').dt.date
cuic = pd.merge(cuic, schedule, left_on=['Employee_ID','Date-1'], right_on =['Emp ID','Date'], how='left')
cuic=cuic[['FullName','LoginName','Interval','AgentAvailTime','AgentLoggedOnTime','Date_x','Time','Employee_ID','Shift_type_x','Shift_x','Date-1','Shift_type_y','Shift_y']]
mytime=t(12,0,0)
condition=(cuic['Shift_type_x']!='DS')&(cuic['Shift_type_y']=='NS')&(cuic['Time']<mytime)
cuic['Session Date'] = np.where(condition, cuic['Date-1'], cuic['Date_x'])
cuic=cuic[['FullName','LoginName','Interval','AgentAvailTime','AgentLoggedOnTime','Date_x','Employee_ID','Shift_type_x','Session Date']]
cuic=cuic.rename(columns={'Date_x':'Date','Shift_type_x':'Shift_type'})
cuic['Session Date'] = pd.to_datetime(cuic['Session Date'], errors='coerce').dt.strftime('%Y-%m-%d')
# export cuic to json
cuic.to_sql('cuic', conn, if_exists='replace', index=False)

C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23636\1492550986.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cuic['Date'] = pd.to_datetime(cuic['Interval'], errors='coerce').dt.date
C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23636\1492550986.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cuic['Time'] = pd.to_datetime(cuic['Interval'], errors='coerce').dt.time


1290391

In [14]:
#EXCEPTION
exceptional_hrs = import_xlsx(path= exception_path, sheet_name="Sheet1")
exceptional_hrs = exceptional_hrs.rename(columns={'Date \n(MM/DD/YYYY)':'Date'})
exceptional_hrs = exceptional_hrs.drop(columns=['filename'])

#export exceptional hours to json
exceptional_hrs.to_sql('exceptional_hrs', conn, if_exists='replace', index=False)


64

In [15]:
#IMPORT IEX ID
iex = import_xlsx(path = iex_path, sheet_name="Sheet1")
iex = iex[['ID Agent', 'Personal ID']]
iex = iex.rename(columns={'Personal ID': 'EID'})

#export iex to json
iex.to_sql('iex', conn, if_exists='replace', index=False)

630

In [16]:
# IMPORT TL
tl = pd.read_excel(os.path.join(tl_path, "TL List.xlsx"),sheet_name='Sheet1')
tl=tl.iloc[:,0:6]
tl=tl.rename(columns={'Supervisor (no need fill)': 'Supervisor'})
tl=tl.drop(columns=['Wave', 'Name', 'LOB', 'Full name (VN)'])

# export tl to json
tl.to_sql('tl', conn, if_exists='replace', index=False)

636

In [17]:
# IMPORT TRAINING
training = import_xlsx(path = training_path,sheet_name='Sheet1')
training['Date'] = pd.to_datetime(training['Date'],format='mixed').dt.date
training['EID'] = training['EID'].astype("int64")
training=training.iloc[:,1:]

#export training to json
training.to_sql('training', conn, if_exists='replace', index=False)

182

In [18]:
# IMPORT IOSHRINKAGE
ioshrinkage = import_xlsx(path = ioshrinkage_path, sheet_name='Sheet1')

#export ioshrinkage to json
ioshrinkage.to_sql('ioshrinkage', conn, if_exists='replace', index=False)

451

In [19]:
# IMPORT OT
if update_less is False:
    ot = import_xlsx(path = ot_full_path, sheet_name='Sheet2')
else:
    ot = import_xlsx(path = ot_path, sheet_name='Sheet2')
ot = ot.drop(columns=['Wave', 'Name', 'Value', 'LOB', 'filename'])
ot['Date'] = pd.to_datetime(ot['Date'], errors='coerce')
ot['Date'] = ot['Date'].dt.date

#export ot to json
ot.to_sql('ot', conn, if_exists='replace', index=False)

57457

In [20]:
# IMPORT PSAT
if update_less is False:
    psat = import_csv(path = psat_full_path)
else:
    psat = import_csv(path = psat_path)

psat['Date'] = pd.to_datetime(psat['Date'],errors='coerce').dt.date

#export psat to json
psat.to_sql('psat', conn, if_exists='replace', index=False)

5630

In [21]:
# IMPORT QUALITY
if update_less is False:
    quality_raw = import_xlsx(path = quality_full_path,sheet_name='Sheet1')
else:
    quality_raw = import_xlsx(path = quality_path,sheet_name='Sheet1')

quality_raw['eval_date'] = pd.to_datetime(quality_raw['eval_date'], errors='coerce').dt.date
quality=quality_raw.iloc[:,0:20]
quality['eval_date'] = pd.to_datetime(quality['eval_date'], errors='coerce').dt.strftime('%Y-%m-%d')
quality['eval_reference'] = quality['eval_id'].astype('str')
#export quality to json
quality.to_sql('quality', conn, if_exists='replace', index=False)


466287

In [22]:
# IMPORT RAMCOOT
if update_less is False:
    ramcoot = import_xlsx(path = ramcoot_full_path,sheet_name='Sheet1')
else:
    ramcoot = import_xlsx(path = ramcoot_path,sheet_name='Sheet1')

ramcoot['Attribute'] = pd.to_datetime(ramcoot['Attribute'],errors='coerce').dt.date
ramcoot=ramcoot.rename(columns={'Attribute':'Date'})

#export ramcoot to json
ramcoot.to_sql('ramcoot', conn, if_exists='replace', index=False)

198095

In [23]:
# IMPORT REQUIREMENT
requirement = import_xlsx(path = requirement_path,sheet_name='Sheet1')
requirement['Date'] = pd.to_datetime(requirement['Date'],errors='coerce').dt.date

#export requirement to json
requirement.to_sql('requirement', conn, if_exists='replace', index=False)

3990

In [24]:
# IMPORT WORKPLAN
workplan_raw = import_xlsx(path = workplan_path,sheet_name='Sheet1')
workplan_raw['Date'] = pd.to_datetime(workplan_raw['Date'],errors='coerce').dt.date
workplan_merge_iex = pd.merge(workplan_raw,iex, left_on='Agent ID', right_on ='ID Agent', how='left')
workplan_merge_iex['EID'] = workplan_merge_iex['EID'].astype("Int64")
workplan_merge_iex=workplan_merge_iex.drop(columns=['filename', 'ID Agent'])
workplan_merge_iex = workplan_merge_iex[['LOB', 'Date', 'Agent ID', 'Agent Name', 'Scheduled Activity', 'Length', 'Percent', 'EID']]
#export workplan_merge_iex to json
workplan_merge_iex.to_sql('workplan_merge_iex', conn, if_exists='replace', index=False)

236904

In [25]:
#IMPORT LABELS
labels = import_csv(label_path)

#export labels to json
labels.to_sql('labels', conn, if_exists='replace', index=False)

34

In [26]:
kpi_target = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\KPI Target\kpi_target.xlsx'), engine="openpyxl", sheet_name = 'Sheet1')
kpi_target.to_sql('kpi_target', conn, if_exists='replace', index=False)

2625

In [27]:
#IMPORT FULL CPI_full
if update_less is False:
    cpi_full = import_csv(path = cpi_full_path)
else:
    cpi_full = import_csv(path = cpi_path)
    
cpi_full = cpi_full.drop(columns=['Region Partner', 'Sitecode', 'Mp Csm Name', 'Mp Team Name'])
cpi_full['Date'] = cpi_full['filename'].replace(".csv", "", regex=True)
cpi_full['Date'] = pd.to_datetime(cpi_full['Date'], errors='coerce')

masterroster_cpi_full = masterroster[['Employee_ID', 'TED Name']]
cpi_full = pd.merge(cpi_full, masterroster_cpi_full, left_on="Staff Name", right_on="TED Name", how="left")
cpi_full = cpi_full.drop(columns=['filename', 'TED Name'])
cpi_full = cpi_full[['Staff Name', 'Employee_ID', 'Date', 'Channel', 'Number of Records']]

# export cpi_full to json
cpi_full.to_sql('cpi_full', conn, if_exists='replace', index=False)

372597

In [28]:
# IMPORT RONA
if update_less is False:
    rona = import_xlsx(path = rona_full_path,sheet_name='RTA-Agent Team Historical All F')
else:
    rona = import_xlsx(path = rona_path,sheet_name='RTA-Agent Team Historical All F')

## extract only 3 cols: Agent, Datetime and RONA
rona = rona[['Agent', 'DateTime', 'RONA']]
## convert time to VN Time
rona['DateTime'] = pd.to_datetime(rona['DateTime'], errors='coerce') + pd.Timedelta(hours=5)
## here we create 2 date and time time cols from DateTime
rona['Date'] = pd.to_datetime(rona['DateTime'], errors='coerce').dt.date
rona['Time'] = pd.to_datetime(rona['DateTime'], errors='coerce').dt.time
## we get the ID from CUIC Name for us to easier manipulating the data
rona = pd.merge(rona, masterroster[['CUIC Name', 'Employee_ID']], left_on='Agent', right_on='CUIC Name', how='left')
rona = rona.drop(columns={'CUIC Name'})
rona['Employee_ID'] = rona['Employee_ID'].dropna()
## Get the Session date
schedule['Date'] = pd.to_datetime(schedule['Date'], errors='coerce').dt.date
rona = pd.merge(rona, schedule[['Emp ID','Date', 'Shift_type']], left_on=['Employee_ID','Date'], right_on =['Emp ID','Date'], how='left')
rona = rona.drop(columns={'Emp ID'})
rona['Date-1']=pd.to_datetime(rona['Date'], errors='coerce') - pd.Timedelta(1,"d")
rona['Date-1']=pd.to_datetime(rona['Date-1'], errors='coerce').dt.date
rona = pd.merge(rona, schedule[['Emp ID', 'Date', 'Shift_type']], left_on=['Employee_ID','Date-1'], right_on =['Emp ID','Date'], how='left')
mytime=t(12,0,0)
condition=(rona['Shift_type_x']!='DS')&(rona['Shift_type_y']=='NS')&(rona['Time']<mytime)
rona['Session Date'] = np.where(condition, rona['Date-1'], rona['Date_x'])
rona = rona.drop(columns={'Date_y', 'Shift_type_x', 'Shift_type_y', 'Emp ID', 'Date-1'})
rona = rona.rename(columns={'Date_x': 'Date'})
rona['Session Date'] = pd.to_datetime(rona['Session Date'], errors='coerce').dt.strftime('%Y-%m-%d')
## extract to json
rona.to_sql('rona', conn, if_exists='replace', index=False)
    

C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23636\2550752670.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rona['DateTime'] = pd.to_datetime(rona['DateTime'], errors='coerce') + pd.Timedelta(hours=5)


1194859

In [29]:
# IMPORT WORKPLAN WITH START TIME
workplan_wst = import_xlsx(path=workplan_wst_path, sheet_name='Sheet1')
# remove rows where there's no activity
workplan_wst = workplan_wst.loc[workplan_wst['Start2'].notna()]

workplan_wst['Date_end'] = workplan_wst.apply(lambda x: pd.to_datetime(x['Date'], errors='coerce') + pd.Timedelta(days=1) if x['Start1'] >= t(20, 0, 0) else pd.to_datetime(x['Date'], errors='coerce'), axis=1)
workplan_wst['date_starttime'] = pd.to_datetime(workplan_wst['Date'], errors='coerce').dt.strftime('%Y-%m-%d') + " " + workplan_wst['Start1'].astype('str')
workplan_wst['date_starttime'] = pd.to_datetime(workplan_wst['date_starttime'], errors='coerce')
workplan_wst['date_endtime'] = pd.to_datetime(workplan_wst['Date_end'], errors='coerce').dt.strftime('%Y-%m-%d') + " " + workplan_wst['End1'].astype('str')
workplan_wst['date_endtime'] = pd.to_datetime(workplan_wst['date_endtime'], errors='coerce')

workplan_wst['date_act_start'] = workplan_wst.apply(lambda x: pd.to_datetime(x['Date'], errors='coerce') + pd.Timedelta(days=1) if x['Start1'] >= t(20, 0, 0) and x['Start2'] <= t(15, 0, 0) else pd.to_datetime(x['Date'], errors='coerce'),axis=1)
workplan_wst['act_starttime'] = pd.to_datetime(workplan_wst['date_act_start'], errors='coerce').dt.strftime('%Y-%m-%d') + " " + workplan_wst['Start2'].astype('str')
workplan_wst['act_starttime'] = pd.to_datetime(workplan_wst['act_starttime'], errors='coerce')
workplan_wst['date_act_end'] = workplan_wst.apply(lambda x: pd.to_datetime(x['Date'], errors='coerce') + pd.Timedelta(days=1) if x['Start1'] >= t(20, 0, 0) and x['End2'] <= t(15, 0, 0) else pd.to_datetime(x['Date'], errors='coerce'),axis=1)
workplan_wst['act_endtime'] = pd.to_datetime(workplan_wst['date_act_end'], errors='coerce').dt.strftime('%Y-%m-%d') + " " + workplan_wst['End2'].astype('str')
workplan_wst['act_endtime'] = pd.to_datetime(workplan_wst['act_endtime'], errors='coerce')

# date_duration show the total time that the agents are workplanned to work in a day
workplan_wst['date_duration'] = (pd.to_datetime(workplan_wst['date_endtime'], format='%Y-%m-%d%H:%M:%S', errors='coerce') - pd.to_datetime(workplan_wst['date_starttime'], format='%Y-%m-%d%H:%M:%S', errors='coerce'))
workplan_wst['date_duration'] = workplan_wst.apply(lambda x: x['date_duration'].total_seconds()/86400, axis=1)

# act_duration show the total time that the agents are workplanned to work in a day
workplan_wst['act_duration'] = (pd.to_datetime(workplan_wst['act_endtime'], format='%Y-%m-%d%H:%M:%S', errors='coerce') - pd.to_datetime(workplan_wst['act_starttime'], format='%Y-%m-%d%H:%M:%S', errors='coerce'))
workplan_wst['act_duration'] = workplan_wst.apply(lambda x: x['act_duration'].total_seconds()/86400, axis=1)

# Trim the table
workplan_wst = workplan_wst[['LOB', 'ID', 'Date', 'date_starttime', 'date_endtime', 'Schedule Act','act_starttime','act_endtime', 'date_duration', 'act_duration']]
# Rename ID to Agent ID
workplan_wst = workplan_wst.rename(columns={'ID': 'Agent ID', 'act_duration': 'Length'})

# Get ID (CNX ID)
workplan_wst = pd.merge(workplan_wst,iex, left_on='Agent ID', right_on ='ID Agent', how='left')
workplan_wst = workplan_wst.drop(columns={'ID Agent'})

# convert datetime value to string and export to json
workplan_wst['Date'] = pd.to_datetime(workplan_wst['Date'], errors='coerce').dt.strftime('%Y-%m-%d')
workplan_wst['date_starttime']=pd.to_datetime(workplan_wst['date_starttime'], errors='coerce').dt.strftime('%Y-%m-%d %I:%M:%S %p')
workplan_wst['date_endtime']=pd.to_datetime(workplan_wst['date_endtime'], errors='coerce').dt.strftime('%Y-%m-%d %I:%M:%S %p')
workplan_wst['act_starttime'] = pd.to_datetime(workplan_wst['act_starttime'], errors='coerce').dt.strftime('%Y-%m-%d %I:%M:%S %p')
workplan_wst['act_endtime'] = pd.to_datetime(workplan_wst['act_endtime'], errors='coerce').dt.strftime('%Y-%m-%d %I:%M:%S %p')

workplan_wst.to_sql('workplan_wst', conn, if_exists='replace', index=False)

128963

In [30]:
conn.commit()
conn.close()